# 🔬 Helmholtz 2D — PINN con campo complejo y LHS

### Proyecto: Simulación Acelerada de Speckle Óptico
**Roberto Hernández Estrada** | Maestría en Ciencias de la Computación — UJAT  
**Director:** Dr. José Adán Hernández Nolasco

---

| Notebook | Estado |
|---|---|
| `01_teoria_pinns.ipynb` — Helmholtz 1D, validación del método | ✅ Completado — Error L2 = 0.002% |
| **`02_helmholtz_2D.ipynb` — Helmholtz 2D, campo complejo, LHS** | 🔄 En desarrollo |
| `03_speckle.ipynb` — Simulación de speckle I = |E|² | 🔜 Pendiente |
| `04_benchmark.ipynb` — PINN vs FEniCSx (Speed-up Factor) | 🔜 Pendiente |

---

## ¿Qué resuelve este notebook?

En el notebook 01 validaste que la PINN puede resolver Helmholtz **1D** con error L2 = 0.002%.  
Este notebook escala al problema **real de la tesis**: Helmholtz en 2 dimensiones.

$$\nabla^2 E + k^2 E = 0 \quad \Longrightarrow \quad 
\frac{\partial^2 E}{\partial x^2} + \frac{\partial^2 E}{\partial y^2} + (2\pi)^2 E = 0$$

La diferencia clave: el campo óptico $E(x,y)$ es **complejo**, así que la PINN ahora predice **dos salidas**:

$$E(x,y) = E_{\text{real}}(x,y) + i \cdot E_{\text{imag}}(x,y)$$

## Diferencias clave respecto al notebook 01

| Aspecto | Notebook 01 (1D) | Notebook 02 (2D) |
|---|---|---|
| Entrada de la red | $x$ (1 valor) | $(x, y)$ (2 valores) |
| Salida de la red | $E$ real (1 valor) | $(E_{\text{real}}, E_{\text{imag}})$ (2 valores) |
| Ecuación | $E'' + k^2 E = 0$ | $\nabla^2 E + k^2 E = 0$ |
| Muestreo interior | Uniforme `linspace` | Latin Hypercube Sampling (LHS) |
| Condiciones de frontera | 5 puntos conocidos | 4 bordes del dominio $[0,1]^2$ |
| Visualización | Gráfica 1D | Mapa de calor 2D (heatmap) |

## ⚙️ 0. Imports y configuración del entorno

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.stats import qmc  # Latin Hypercube Sampling

# ── Semilla global — SIEMPRE al inicio para reproducibilidad ──────────────────
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

# ── Dispositivo ───────────────────────────────────────────────────────────────
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Dispositivo     : {device}')
if torch.cuda.is_available():
    print(f'GPU             : {torch.cuda.get_device_name(0)}')
    print(f'VRAM total      : {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
print(f'Semilla fijada  : {SEED}')
print()
print('✅ Entorno listo y reproducible.')

---
## 📌 1. Arquitectura de la red — PINN 2D

La red ahora recibe dos coordenadas $(x, y)$ y predice dos salidas $(E_{\text{real}}, E_{\text{imag}})$.  
La estructura interna es idéntica al notebook 01 — mismo número de capas, misma activación `tanh`, misma inicialización Xavier.

### 💡 ¿Por qué dos salidas?

El campo eléctrico óptico es una cantidad **compleja**:

$$E(x,y) = E_{\text{real}}(x,y) + i \cdot E_{\text{imag}}(x,y)$$

Las redes neuronales reales solo manejan números reales, así que la estrategia estándar en PINNs ópticas es entrenar **una sola red con dos salidas**, una por cada parte del campo.

Ambas partes satisfacen la ecuación de Helmholtz de forma independiente:

$$\nabla^2 E_{\text{real}} + k^2 E_{\text{real}} = 0$$
$$\nabla^2 E_{\text{imag}} + k^2 E_{\text{imag}} = 0$$

La intensidad del speckle (lo que mide el detector) se obtiene después:

$$I(x,y) = |E|^2 = E_{\text{real}}^2 + E_{\text{imag}}^2$$

In [ ]:
class PINN_2D(nn.Module):
    """
    Red neuronal para resolver Helmholtz 2D: ∇²E + k²E = 0

    Entrada : (x, y) — coordenadas espaciales 2D
    Salida  : (E_real, E_imag) — partes real e imaginaria del campo
    """
    def __init__(self, hidden_dim=64, num_layers=5):
        super().__init__()

        # Entrada: 2 neuronas (x, y) → misma arquitectura que 1D internamente
        layers = [nn.Linear(2, hidden_dim), nn.Tanh()]
        for _ in range(num_layers - 1):
            layers += [nn.Linear(hidden_dim, hidden_dim), nn.Tanh()]
        layers.append(nn.Linear(hidden_dim, 2))  # 2 salidas: E_real, E_imag

        self.net = nn.Sequential(*layers)
        self._init_weights()

    def _init_weights(self):
        """Inicialización Xavier — misma estrategia que en PINN_1D."""
        for layer in self.net:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_normal_(layer.weight)
                nn.init.zeros_(layer.bias)

    def forward(self, xy):
        """
        xy : tensor de forma (N, 2) con columnas [x, y]
        Retorna tensor de forma (N, 2) con columnas [E_real, E_imag]
        """
        return self.net(xy)


# ── Vista previa de la arquitectura ───────────────────────────────────────────
torch.manual_seed(SEED)
_preview = PINN_2D(hidden_dim=64, num_layers=5)
print('Arquitectura PINN 2D:')
print(_preview)
print(f'\nTotal de parámetros: {sum(p.numel() for p in _preview.parameters()):,}')
del _preview

---
## 📌 2. Muestreo LHS — Latin Hypercube Sampling

En 1D usamos `linspace` para distribuir puntos uniformemente.  
En 2D necesitamos una estrategia más eficiente: **Latin Hypercube Sampling**.

### 💡 ¿Por qué LHS en lugar de una malla uniforme?

Con una malla uniforme de $N \times N$ puntos en 2D, el número de puntos crece cuadráticamente.  
Para $N=45$ ya son 2,025 puntos — y muchos de ellos quedan agrupados en regiones redundantes.

LHS garantiza que los puntos estén **bien distribuidos en todo el dominio** con menos puntos:

| Muestreo | 2000 puntos en [0,1]² | Cobertura | Costo |
|---|---|---|---|
| Malla uniforme | 44×44 = 1,936 pts | Regular pero redundante | Fijo |
| Aleatorio puro | 2000 pts | Irregular, huecos | Variable |
| **LHS** | **2000 pts** | **Uniforme garantizada** | **Controlado** |

LHS divide cada dimensión en $N$ estratos iguales y coloca exactamente un punto en cada estrato,  
asegurando cobertura completa del dominio sin agrupar puntos.

---
## 📌 3. Configuración del experimento

Misma filosofía que en el notebook 01: todos los hiperparámetros en un solo lugar.  
El dominio es ahora $[0,1]^2$ — equivalente a $1\lambda \times 1\lambda$ (Fase 2, validación 2D).

### 💡 Solución analítica en 2D

Para validar la PINN necesitamos una solución exacta conocida.  
Usamos una onda plana que satisface Helmholtz 2D exactamente:

$$E_{\text{real}}(x,y) = \cos(k_x x + k_y y), \quad E_{\text{imag}}(x,y) = \sin(k_x x + k_y y)$$

con $k_x^2 + k_y^2 = k^2 = (2\pi)^2$.

La elección más simple y simétrica:

$$k_x = k_y = \frac{k}{\sqrt{2}} = \frac{2\pi}{\sqrt{2}} = \pi\sqrt{2}$$

**Verificación:** $k_x^2 + k_y^2 = 2\pi^2 + 2\pi^2 = 4\pi^2 = k^2$ ✅

In [ ]:
# ── Hiperparámetros ────────────────────────────────────────────────────────────
CONFIG = {
    # ── Láser simulado (documentación — no entra en cálculos de la PINN) ──────
    'lambda_laser' : 638e-9,      # diodo rojo λ = 638 nm (computacional)
    'L'            : 1,           # dominio = 1×1 longitudes de onda (Fase 2)

    # ── Física adimensional ────────────────────────────────────────────────────
    'k'            : 2 * np.pi,   # k̃ adimensional — siempre 2π
                                   # k_x = k_y = k/sqrt(2) → onda plana diagonal

    # ── Puntos de colocación (LHS) ─────────────────────────────────────────────
    'N_colloc'     : 2000,        # puntos interiores — Latin Hypercube Sampling
    'N_boundary'   : 200,         # puntos por borde (4 bordes → 800 total)

    # ── Arquitectura ───────────────────────────────────────────────────────────
    'hidden_dim'   : 64,          # neuronas por capa oculta
    'num_layers'   : 5,           # capas ocultas

    # ── Entrenamiento ──────────────────────────────────────────────────────────
    'n_epochs'     : 15000,       # épocas máximas Adam
    'lr'           : 1e-3,        # learning rate Adam
    'lambda_phys'  : 0.01,        # peso física — mismo valor validado en 1D

    # ── Reproducibilidad ───────────────────────────────────────────────────────
    'seed'         : SEED,

    # ── Meta de la tesis ───────────────────────────────────────────────────────
    'l2_threshold' : 0.05,        # < 5% según hipótesis del protocolo
}

print('Configuración del experimento:')
print('-' * 40)
for key, val in CONFIG.items():
    print(f'  {key:<16} : {val}')
print('-' * 40)

# ── Derivar k_x, k_y ──────────────────────────────────────────────────────────
k    = CONFIG['k']
k_x  = k / np.sqrt(2)
k_y  = k / np.sqrt(2)
print(f'  k_x = k/√2       : {k_x:.4f}')
print(f'  k_y = k/√2       : {k_y:.4f}')
print(f'  Verificación k²  : k_x²+k_y² = {k_x**2 + k_y**2:.4f} = k² = {k**2:.4f} ✅')
print()

# ── Equivalencia física ────────────────────────────────────────────────────────
lam = CONFIG['lambda_laser']
L   = CONFIG['L']
print('Láser simulado:')
print(f'  Tipo                : Diodo rojo (computacional)')
print(f'  lambda_laser        : {lam * 1e9:.0f} nm')
print(f'  k adimensional      : 2π = {k:.4f}')
print(f'  k real equivalente  : {2 * np.pi / lam:.3e} m⁻¹')
print(f'  Dominio simulado    : {L}×{L} longitudes de onda')
print(f'  Equivale físicamente: {L * lam * 1e6:.4f} × {L * lam * 1e6:.4f} μm²')

---
## 📌 4. Preparar datos de entrenamiento

### Puntos de colocación — LHS en el interior
### Condiciones de frontera — 4 bordes del dominio $[0,1]^2$

### 💡 La función de pérdida PINN 2D

La pérdida total combina tres contribuciones — una por cada parte del campo y los datos:

$$\mathcal{L}_{\text{total}} = \mathcal{L}_{\text{datos}} + \lambda \cdot (\mathcal{L}_{\text{física,real}} + \mathcal{L}_{\text{física,imag}})$$

donde:

$$\mathcal{L}_{\text{datos}} = \frac{1}{N_b} \sum_{i=1}^{N_b} \left[(E^{\text{real}}_{\text{pred},i} - E^{\text{real}}_{\text{BC},i})^2 + (E^{\text{imag}}_{\text{pred},i} - E^{\text{imag}}_{\text{BC},i})^2\right]$$

$$\mathcal{L}_{\text{física,real}} = \frac{1}{N_c} \sum_{j=1}^{N_c} \left[\nabla^2 E^{\text{real}}_j + k^2 E^{\text{real}}_j\right]^2$$

In [ ]:
k   = CONFIG['k']
k_x = k / np.sqrt(2)
k_y = k / np.sqrt(2)

# ── Puntos interiores con LHS ──────────────────────────────────────────────────
sampler   = qmc.LatinHypercube(d=2, seed=CONFIG['seed'])
lhs_pts   = sampler.random(n=CONFIG['N_colloc'])          # (N, 2) en [0,1]²
xy_colloc = torch.tensor(lhs_pts, dtype=torch.float32).to(device)

print(f'Puntos de colocación (LHS): {xy_colloc.shape}  → {CONFIG["N_colloc"]} puntos en [0,1]²')

# ── Condiciones de frontera — 4 bordes ────────────────────────────────────────
N_b  = CONFIG['N_boundary']
t    = np.linspace(0, 1, N_b)

# Borde inferior: y=0, x∈[0,1]
x_bot = np.stack([t,        np.zeros(N_b)], axis=1)
# Borde superior: y=1, x∈[0,1]
x_top = np.stack([t,        np.ones(N_b)],  axis=1)
# Borde izquierdo: x=0, y∈[0,1]
x_lft = np.stack([np.zeros(N_b), t],        axis=1)
# Borde derecho: x=1, y∈[0,1]
x_rgt = np.stack([np.ones(N_b),  t],        axis=1)

xy_bc_np = np.vstack([x_bot, x_top, x_lft, x_rgt])   # (4*N_b, 2)

# ── Solución analítica en los bordes ──────────────────────────────────────────
# E_real(x,y) = cos(k_x·x + k_y·y)
# E_imag(x,y) = sin(k_x·x + k_y·y)
phase_bc      = k_x * xy_bc_np[:, 0] + k_y * xy_bc_np[:, 1]
E_real_bc_np  = np.cos(phase_bc)
E_imag_bc_np  = np.sin(phase_bc)
E_bc_np       = np.stack([E_real_bc_np, E_imag_bc_np], axis=1)  # (4*N_b, 2)

# ── Convertir a tensores ───────────────────────────────────────────────────────
xy_bc = torch.tensor(xy_bc_np, dtype=torch.float32).to(device)
E_bc  = torch.tensor(E_bc_np,  dtype=torch.float32).to(device)

print(f'Puntos de frontera         : {xy_bc.shape}  → {4*N_b} puntos ({N_b} por borde)')
print(f'  Borde inferior (y=0)     : {N_b} puntos')
print(f'  Borde superior (y=1)     : {N_b} puntos')
print(f'  Borde izquierdo (x=0)    : {N_b} puntos')
print(f'  Borde derecho (x=1)      : {N_b} puntos')
print()
print('Muestra de condiciones de frontera:')
print(f'  E(0,0)   real={E_real_bc_np[0]:.4f}   imag={E_imag_bc_np[0]:.4f}')
print(f'  E(0.5,0) real={np.cos(k_x*0.5):.4f}   imag={np.sin(k_x*0.5):.4f}')
print(f'  E(1,1)   real={np.cos(k_x+k_y):.4f}   imag={np.sin(k_x+k_y):.4f}')

---
## 📌 5. Residuo de Helmholtz 2D y función de pérdida

In [ ]:
def helmholtz_residual_2d(model, xy_colloc, k):
    """
    Calcula el residuo de Helmholtz 2D para ambas partes del campo:
        R_real = ∂²E_real/∂x² + ∂²E_real/∂y² + k²·E_real
        R_imag = ∂²E_imag/∂x² + ∂²E_imag/∂y² + k²·E_imag

    Si la red es perfecta → R_real = R_imag = 0 en todo el dominio.
    """
    xy = xy_colloc.clone().requires_grad_(True)

    E_out  = model(xy)                        # (N, 2)
    E_real = E_out[:, 0:1]                    # (N, 1)
    E_imag = E_out[:, 1:2]                    # (N, 1)

    ones = torch.ones_like(E_real)

    def laplacian(field):
        """Calcula ∂²f/∂x² + ∂²f/∂y² usando autograd."""
        grad_f  = torch.autograd.grad(field, xy, grad_outputs=ones,
                                      create_graph=True)[0]          # (N, 2)
        # ∂²/∂x²
        f_xx = torch.autograd.grad(grad_f[:, 0:1], xy, grad_outputs=ones,
                                   create_graph=True)[0][:, 0:1]
        # ∂²/∂y²
        f_yy = torch.autograd.grad(grad_f[:, 1:2], xy, grad_outputs=ones,
                                   create_graph=True)[0][:, 1:2]
        return f_xx + f_yy

    lap_real = laplacian(E_real)
    lap_imag = laplacian(E_imag)

    res_real = lap_real + k**2 * E_real
    res_imag = lap_imag + k**2 * E_imag

    return res_real, res_imag, E_real, E_imag


def pinn_loss_2d(model, xy_colloc, xy_bc, E_bc, k, lambda_phys):
    """
    Función de pérdida PINN 2D:
        L_total = L_datos + lambda * (L_física_real + L_física_imag)
    """
    # ── Pérdida de datos (condiciones de frontera) ─────────────────────────────
    E_pred_bc  = model(xy_bc)                            # (N_b, 2)
    loss_data  = torch.mean((E_pred_bc - E_bc) ** 2)

    # ── Pérdida física (residuo Helmholtz) ─────────────────────────────────────
    res_real, res_imag, _, _ = helmholtz_residual_2d(model, xy_colloc, k)
    loss_phys_real = torch.mean(res_real ** 2)
    loss_phys_imag = torch.mean(res_imag ** 2)
    loss_physics   = loss_phys_real + loss_phys_imag

    # ── Total ──────────────────────────────────────────────────────────────────
    loss_total = loss_data + lambda_phys * loss_physics

    return loss_total, loss_data, loss_physics


print('✅ Funciones auxiliares 2D definidas.')
print('   helmholtz_residual_2d() — calcula ∇²E_real + k²E_real  y  ∇²E_imag + k²E_imag')
print('   pinn_loss_2d()          — calcula L_total = L_datos + λ·(L_real + L_imag)')

---
## 📌 6. Entrenamiento — Adam + Early Stopping + L-BFGS

### 💡 Misma estrategia de entrenamiento que en 1D

El notebook 01 demostró que la combinación **Adam + Early Stopping + L-BFGS** es óptima:

| Fase | Optimizador | Rol |
|---|---|---|
| 1 | Adam | Exploración rápida del espacio de parámetros |
| — | Early Stopping | Detiene Adam cuando $\mathcal{L} < 10^{-5}$ — evita sobreajuste |
| 2 | L-BFGS | Ajuste fino de precisión cerca del mínimo |

Los mismos hiperparámetros validados en 1D (`lambda_phys=0.01`, `lr=1e-3`) se mantienen como punto de partida.

In [ ]:
# ── Semilla antes de instanciar ───────────────────────────────────────────────
torch.manual_seed(CONFIG['seed'])
torch.cuda.manual_seed(CONFIG['seed'])

# ── Modelo ────────────────────────────────────────────────────────────────────
model_2d = PINN_2D(
    hidden_dim = CONFIG['hidden_dim'],
    num_layers = CONFIG['num_layers']
).to(device)

optimizer = torch.optim.Adam(model_2d.parameters(), lr=CONFIG['lr'])

# ── Loop de entrenamiento Adam con Early Stopping ─────────────────────────────
history = {'total': [], 'data': [], 'physics': []}

ADAM_STOP_THRESHOLD = 1e-5

print('Entrenando PINN 2D...')
print(f'  k={k:.4f} | LHS {CONFIG["N_colloc"]} pts | {4*CONFIG["N_boundary"]} pts frontera')
print(f'  Épocas máx: {CONFIG["n_epochs"]} | lr={CONFIG["lr"]} | λ={CONFIG["lambda_phys"]}')
print(f'  Early stopping: L_total < {ADAM_STOP_THRESHOLD:.0e}')
print()

epoch_final = CONFIG['n_epochs']

for epoch in range(CONFIG['n_epochs']):
    optimizer.zero_grad()

    loss, loss_data, loss_physics = pinn_loss_2d(
        model_2d, xy_colloc, xy_bc, E_bc,
        k           = CONFIG['k'],
        lambda_phys = CONFIG['lambda_phys']
    )

    loss.backward()
    optimizer.step()

    history['total'].append(loss.item())
    history['data'].append(loss_data.item())
    history['physics'].append(loss_physics.item())

    if (epoch + 1) % 500 == 0:
        print(f'  Época {epoch+1:5d} | '
              f'L_total: {loss.item():.3e} | '
              f'L_datos: {loss_data.item():.3e} | '
              f'L_física: {loss_physics.item():.3e}')

    if loss.item() < ADAM_STOP_THRESHOLD:
        epoch_final = epoch + 1
        print(f'\n  ⏹ Early stopping en época {epoch_final} — '
              f'L_total={loss.item():.3e} < {ADAM_STOP_THRESHOLD:.0e}')
        print(f'  Épocas ahorradas: {CONFIG["n_epochs"] - epoch_final}')
        break

print(f'\n✅ Adam completado en {epoch_final} épocas.')

# ── Ajuste fino con L-BFGS ────────────────────────────────────────────────────
print('\nAjuste fino con L-BFGS...')

optimizer_lbfgs = torch.optim.LBFGS(
    model_2d.parameters(),
    lr             = 1.0,
    max_iter       = 500,
    history_size   = 50,
    line_search_fn = 'strong_wolfe'
)

history_lbfgs = {'total': [], 'data': [], 'physics': []}
iter_count = [0]

def closure():
    optimizer_lbfgs.zero_grad()
    loss, loss_data, loss_physics = pinn_loss_2d(
        model_2d, xy_colloc, xy_bc, E_bc,
        k           = CONFIG['k'],
        lambda_phys = CONFIG['lambda_phys']
    )
    loss.backward()
    history_lbfgs['total'].append(loss.item())
    history_lbfgs['data'].append(loss_data.item())
    history_lbfgs['physics'].append(loss_physics.item())
    iter_count[0] += 1
    if iter_count[0] % 100 == 0:
        print(f'  Iter {iter_count[0]:4d} | '
              f'L_total: {loss.item():.3e} | '
              f'L_datos: {loss_data.item():.3e} | '
              f'L_física: {loss_physics.item():.3e}')
    return loss

optimizer_lbfgs.step(closure)

print('\n✅ L-BFGS completado.')
print(f'   Épocas Adam     : {epoch_final} / {CONFIG["n_epochs"]}')
print(f'   Épocas ahorradas: {CONFIG["n_epochs"] - epoch_final}')
print(f'   Iters L-BFGS    : {iter_count[0]}')

---
## 📌 7. Evaluación y visualización de resultados

Evaluamos la PINN en una malla uniforme de $100 \times 100$ puntos (10,000 puntos de prueba)  
y comparamos contra la solución analítica exacta.

In [ ]:
# ── Malla de evaluación 100×100 ───────────────────────────────────────────────
N_eval = 100
x_lin  = np.linspace(0, 1, N_eval)
y_lin  = np.linspace(0, 1, N_eval)
XX, YY = np.meshgrid(x_lin, y_lin)

xy_test_np = np.stack([XX.ravel(), YY.ravel()], axis=1)  # (10000, 2)
xy_test    = torch.tensor(xy_test_np, dtype=torch.float32).to(device)

# ── Predicción ────────────────────────────────────────────────────────────────
model_2d.eval()
with torch.no_grad():
    E_pred = model_2d(xy_test).cpu().numpy()   # (10000, 2)

E_real_pred = E_pred[:, 0].reshape(N_eval, N_eval)
E_imag_pred = E_pred[:, 1].reshape(N_eval, N_eval)

# ── Solución analítica ────────────────────────────────────────────────────────
phase_test   = k_x * XX + k_y * YY
E_real_exact = np.cos(phase_test)
E_imag_exact = np.sin(phase_test)
I_exact      = E_real_exact**2 + E_imag_exact**2  # = 1 para onda plana

# ── Intensidad PINN ───────────────────────────────────────────────────────────
I_pred = E_real_pred**2 + E_imag_pred**2

# ── Métricas ──────────────────────────────────────────────────────────────────
def l2_rel(pred, exact):
    return np.linalg.norm(pred.ravel() - exact.ravel()) / np.linalg.norm(exact.ravel())

l2_real = l2_rel(E_real_pred, E_real_exact)
l2_imag = l2_rel(E_imag_pred, E_imag_exact)
l2_mean = (l2_real + l2_imag) / 2
cumple  = l2_mean < CONFIG['l2_threshold']

error_real = np.abs(E_real_pred - E_real_exact)
error_imag = np.abs(E_imag_pred - E_imag_exact)

print(f'Métricas de evaluación (malla {N_eval}×{N_eval}):')
print(f'  Error L2 E_real   : {l2_real*100:.3f}%')
print(f'  Error L2 E_imag   : {l2_imag*100:.3f}%')
print(f'  Error L2 promedio : {l2_mean*100:.3f}%')
print(f'  Umbral tesis      : < {CONFIG["l2_threshold"]*100:.0f}%')
print(f'  Estado            : {"✅ CUMPLE" if cumple else "⚠ AÚN NO CUMPLE"}'  )

In [ ]:
# ── Visualización ─────────────────────────────────────────────────────────────
fig = plt.figure(figsize=(18, 14))
gs  = gridspec.GridSpec(3, 3, figure=fig, hspace=0.45, wspace=0.35)

cmap_field = 'RdBu_r'
cmap_error = 'hot_r'

# ── Fila 1: E_real ────────────────────────────────────────────────────────────
ax = fig.add_subplot(gs[0, 0])
im = ax.imshow(E_real_exact, origin='lower', extent=[0,1,0,1],
               cmap=cmap_field, vmin=-1, vmax=1)
ax.set_title('E_real — Solución exacta', fontsize=11, fontweight='bold')
ax.set_xlabel('x'); ax.set_ylabel('y')
plt.colorbar(im, ax=ax)

ax = fig.add_subplot(gs[0, 1])
im = ax.imshow(E_real_pred, origin='lower', extent=[0,1,0,1],
               cmap=cmap_field, vmin=-1, vmax=1)
ax.set_title('E_real — Predicción PINN', fontsize=11, fontweight='bold')
ax.set_xlabel('x'); ax.set_ylabel('y')
plt.colorbar(im, ax=ax)

ax = fig.add_subplot(gs[0, 2])
im = ax.imshow(error_real, origin='lower', extent=[0,1,0,1],
               cmap=cmap_error)
ax.set_title(f'|Error| E_real — L2 = {l2_real*100:.3f}%', fontsize=11, fontweight='bold')
ax.set_xlabel('x'); ax.set_ylabel('y')
plt.colorbar(im, ax=ax)

# ── Fila 2: E_imag ────────────────────────────────────────────────────────────
ax = fig.add_subplot(gs[1, 0])
im = ax.imshow(E_imag_exact, origin='lower', extent=[0,1,0,1],
               cmap=cmap_field, vmin=-1, vmax=1)
ax.set_title('E_imag — Solución exacta', fontsize=11, fontweight='bold')
ax.set_xlabel('x'); ax.set_ylabel('y')
plt.colorbar(im, ax=ax)

ax = fig.add_subplot(gs[1, 1])
im = ax.imshow(E_imag_pred, origin='lower', extent=[0,1,0,1],
               cmap=cmap_field, vmin=-1, vmax=1)
ax.set_title('E_imag — Predicción PINN', fontsize=11, fontweight='bold')
ax.set_xlabel('x'); ax.set_ylabel('y')
plt.colorbar(im, ax=ax)

ax = fig.add_subplot(gs[1, 2])
im = ax.imshow(error_imag, origin='lower', extent=[0,1,0,1],
               cmap=cmap_error)
ax.set_title(f'|Error| E_imag — L2 = {l2_imag*100:.3f}%', fontsize=11, fontweight='bold')
ax.set_xlabel('x'); ax.set_ylabel('y')
plt.colorbar(im, ax=ax)

# ── Fila 3: Intensidad y pérdida ──────────────────────────────────────────────
ax = fig.add_subplot(gs[2, 0])
im = ax.imshow(I_pred, origin='lower', extent=[0,1,0,1], cmap='inferno')
ax.set_title('Intensidad PINN: I = |E|²', fontsize=11, fontweight='bold')
ax.set_xlabel('x'); ax.set_ylabel('y')
plt.colorbar(im, ax=ax)

# Distribución LHS
ax = fig.add_subplot(gs[2, 1])
xy_lhs_np = xy_colloc.cpu().numpy()
ax.scatter(xy_lhs_np[:, 0], xy_lhs_np[:, 1], s=1, alpha=0.4, c='steelblue')
ax.set_title(f'Puntos LHS ({CONFIG["N_colloc"]} pts)', fontsize=11, fontweight='bold')
ax.set_xlabel('x'); ax.set_ylabel('y')
ax.set_xlim(0, 1); ax.set_ylim(0, 1)
ax.set_aspect('equal')

# Curvas de pérdida
ax = fig.add_subplot(gs[2, 2])
ep_adam  = range(1, epoch_final + 1)
ep_lbfgs = range(epoch_final, epoch_final + len(history_lbfgs['total']))
ax.semilogy(ep_adam, history['total'][:epoch_final],   'k-',  lw=2,   label='L_total (Adam)')
ax.semilogy(ep_adam, history['data'][:epoch_final],    'b--', lw=1.5, label='L_datos (Adam)')
ax.semilogy(ep_adam, history['physics'][:epoch_final], 'r--', lw=1.5, label='L_física (Adam)')
ax.semilogy(ep_lbfgs, history_lbfgs['total'],   'k-',  lw=2,   alpha=0.4, label='L_total (L-BFGS)')
ax.semilogy(ep_lbfgs, history_lbfgs['data'],    'b--', lw=1.5, alpha=0.4, label='L_datos (L-BFGS)')
ax.semilogy(ep_lbfgs, history_lbfgs['physics'], 'r--', lw=1.5, alpha=0.4, label='L_física (L-BFGS)')
ax.axvline(x=epoch_final, color='gray', ls=':', lw=2, label=f'Early stop é.{epoch_final}')
ax.set_title('Curvas de pérdida', fontsize=11, fontweight='bold')
ax.set_xlabel('Época / Iteración'); ax.set_ylabel('Loss (log)')
ax.legend(fontsize=8); ax.grid(True, alpha=0.3)

plt.suptitle(
    f'PINN 2D — Helmholtz | k=2π | Error L2 promedio = {l2_mean*100:.3f}% | '
    f'Meta < {CONFIG["l2_threshold"]*100:.0f}%',
    fontsize=14, fontweight='bold'
)
plt.savefig('resultados_pinn_2d.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── Resumen final ─────────────────────────────────────────────────────────────
print('=' * 55)
print('RESUMEN DE RESULTADOS — PINN 2D')
print('=' * 55)
print(f'  Semilla               : {CONFIG["seed"]}')
print(f'  k adimensional        : 2π = {k:.4f}')
print(f'  k_x = k_y             : {k_x:.4f}')
print()
print(f'  Error L2 E_real       : {l2_real:.6f}  ({l2_real*100:.3f}%)')
print(f'  Error L2 E_imag       : {l2_imag:.6f}  ({l2_imag*100:.3f}%)')
print(f'  Error L2 promedio     : {l2_mean:.6f}  ({l2_mean*100:.3f}%)')
print(f'  Error máx E_real      : {error_real.max():.6f}')
print(f'  Error máx E_imag      : {error_imag.max():.6f}')
print(f'  Umbral tesis (L2)     : < {CONFIG["l2_threshold"]*100:.0f}%')
print(f'  Estado                : {"✅ CUMPLE" if cumple else "⚠ AÚN NO CUMPLE — ajustar hiperparámetros"}')
print()
print('Entrenamiento:')
print(f'  Épocas Adam           : {epoch_final} / {CONFIG["n_epochs"]}')
print(f'  Épocas ahorradas      : {CONFIG["n_epochs"] - epoch_final}')
print(f'  Iteraciones L-BFGS    : {iter_count[0]}')
print()
print('Hiperparámetros:')
print(f'  hidden_dim            = {CONFIG["hidden_dim"]}')
print(f'  num_layers            = {CONFIG["num_layers"]}')
print(f'  N_colloc (LHS)        = {CONFIG["N_colloc"]}')
print(f'  N_boundary (por borde)= {CONFIG["N_boundary"]}  → {4*CONFIG["N_boundary"]} total')
print(f'  lambda_phys           = {CONFIG["lambda_phys"]}')
print()
lam = CONFIG['lambda_laser']
L   = CONFIG['L']
print('Láser simulado:')
print(f'  Tipo                  : Diodo rojo (computacional)')
print(f'  lambda_laser          : {lam * 1e9:.0f} nm')
print(f'  k adimensional        : 2π = {k:.4f}')
print(f'  k real equivalente    : {2 * np.pi / lam:.3e} m⁻¹')
print(f'  Dominio simulado      : {L}×{L} longitudes de onda')
print(f'  Equivale físicamente  : {L * lam * 1e6:.4f} × {L * lam * 1e6:.4f} μm²')

---
## 🗺️ ¿Qué sigue?

Con este notebook validaste que la PINN resuelve Helmholtz **2D** con campo complejo.  
El siguiente paso genera el **patrón de speckle óptico** — el objetivo central de la tesis:

| Notebook | Contenido |
|---|---|
| `01_teoria_pinns.ipynb` ✅ | Helmholtz 1D — validación del método |
| **`02_helmholtz_2D.ipynb` ✅** | **Helmholtz 2D — campo complejo, LHS** |
| `03_speckle.ipynb` 🔜 | $I(x,y) = |E|^2$ — patrón de speckle + estadística |
| `04_benchmark.ipynb` 🔜 | PINN vs FEniCSx — Speed-up Factor |

## 📋 Lecciones aprendidas en 2D

| Aspecto | Decisión |
|---|---|
| Campo complejo | Una red, dos salidas: $(E_{\text{real}}, E_{\text{imag}})$ |
| Muestreo interior | LHS — cobertura uniforme garantizada |
| Condiciones de frontera | 4 bordes completos, $N_b$ puntos por borde |
| Residuo Helmholtz | Laplaciano vía autograd en dos dimensiones |
| Pérdida | $\mathcal{L} = \mathcal{L}_{\text{datos}} + \lambda(\mathcal{L}_{\text{real}} + \mathcal{L}_{\text{imag}})$ |
| Estrategia | Adam + Early Stopping + L-BFGS — misma que 1D |

> **Nota:** El notebook 03 tomará la predicción $E(x,y)$ de este notebook  
> y calculará $I = |E|^2$ para generar y analizar el patrón de speckle.